In [3]:
import requests


In [15]:
headers = {
  'Accept': 'application/json'
}
 
r = requests.get('https://api.carbonintensity.org.uk/intensity/{from}/fw24h', params={"from":"2017-08-25T12:35Z"}, headers = headers)

print(r.json())

{'error': {'code': '400 Bad Request', 'message': 'Please enter a valid datetime in ISO8601 format YYYY-MM-DDThh:mmZ e.g. /intensity/2017-08-25T15:30Z/fw24h'}}


In [40]:
r = requests.get('https://api.carbonintensity.org.uk/intensity/2022-10-22T12:35Z/fw24h', headers = headers)
r = r.json()

In [41]:
print(r)

{'data': [{'from': '2022-10-22T12:30Z', 'to': '2022-10-22T13:00Z', 'intensity': {'forecast': 156, 'actual': 167, 'index': 'moderate'}}, {'from': '2022-10-22T13:00Z', 'to': '2022-10-22T13:30Z', 'intensity': {'forecast': 162, 'actual': 174, 'index': 'moderate'}}, {'from': '2022-10-22T13:30Z', 'to': '2022-10-22T14:00Z', 'intensity': {'forecast': 163, 'actual': 185, 'index': 'moderate'}}, {'from': '2022-10-22T14:00Z', 'to': '2022-10-22T14:30Z', 'intensity': {'forecast': 168, 'actual': 198, 'index': 'moderate'}}, {'from': '2022-10-22T14:30Z', 'to': '2022-10-22T15:00Z', 'intensity': {'forecast': 187, 'actual': 203, 'index': 'moderate'}}, {'from': '2022-10-22T15:00Z', 'to': '2022-10-22T15:30Z', 'intensity': {'forecast': 207, 'actual': None, 'index': 'moderate'}}, {'from': '2022-10-22T15:30Z', 'to': '2022-10-22T16:00Z', 'intensity': {'forecast': 216, 'actual': None, 'index': 'high'}}, {'from': '2022-10-22T16:00Z', 'to': '2022-10-22T16:30Z', 'intensity': {'forecast': 230, 'actual': None, 'index

In [29]:
import pandas as pd

In [49]:
data = pd.json_normalize(r['data'], max_level=1)
data = data.drop(['to', 'intensity.actual', 'intensity.index'], axis = 1)
data.set_index('from')

,intensity.forecast
from,
2022-10-22T12:30Z,156
2022-10-22T13:00Z,162
2022-10-22T13:30Z,163
2022-10-22T14:00Z,168
2022-10-22T14:30Z,187
2022-10-22T15:00Z,207
2022-10-22T15:30Z,216
2022-10-22T16:00Z,230
2022-10-22T16:30Z,244
